<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Распознавание-адресов" data-toc-modified-id="Распознавание-адресов-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Распознавание адресов</a></span></li><li><span><a href="#Строки-с-адресами-за-пределами-Липецкой-области-или-без-адреса" data-toc-modified-id="Строки-с-адресами-за-пределами-Липецкой-области-или-без-адреса-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Строки с адресами за пределами Липецкой области или без адреса</a></span></li><li><span><a href="#Финальная-обработка-названий-районов" data-toc-modified-id="Финальная-обработка-названий-районов-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Финальная обработка названий районов</a></span></li></ul></div>

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd

from datetime import datetime
from sqlalchemy import create_engine

import ipywidgets as widgets
import time

In [2]:
POSTGRES_ADDRESS = '10.248.23.152'
POSTGRES_PORT = '5432'
POSTGRES_USERNAME = 'isu'
POSTGRES_PASSWORD = 'isupass'
POSTGRES_DBNAME = 'isu_db'

postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(username=POSTGRES_USERNAME, password=POSTGRES_PASSWORD, ipaddress=POSTGRES_ADDRESS, port=POSTGRES_PORT, dbname=POSTGRES_DBNAME))

cnx = create_engine(postgres_str)

In [3]:
df = pd.read_sql_query('''select * from fertility''', cnx)
df[['mother_year_of_birth', 'due_date_year',
    'due_date_month', 'mother_age']] = df[['mother_year_of_birth', 'due_date_year',
                                           'due_date_month', 'mother_age']].astype('int64')
print(f'Количество записей: {len(df)}')

Количество записей: 43367


In [4]:
mo_dict = {'липецк ': 'Липецк', 
           'елец ': 'Елец',           
           'грязинский': 'Грязинский', 'липецкий': 'Липецкий', 'усманский': 'Усманский', 'лебедянский': 'Лебедянский', 
           'задонский': 'Задонский', 'добринский': 'Добринский', 'данковский': 'Данковский', 
           'чаплыгинский': 'Чаплыгинский', 'елецкий': 'Елецкий', 'добровский': 'Добровский', 'тербунский': 'Тербунский', 
           'хлевенский': 'Хлевенский', 'становлянский': 'Становлянский', 'долгоруковский': 'Долгоруковский',  
           'лев-толстовский': 'Лев-Толстовский', 'измалковский': 'Измалковский', 'воловский': 'Воловский',  
           'краснинский': 'Краснинский',  
           'липецк,': 'Липецк', 'грязи': 'Грязинский', 'усмань': 'Усманский', 'лебедянь': 'Лебедянский', 
           'задонск': 'Задонский', 'добринка': 'Добринский', 'данков': 'Данковский', 'чаплыгин': 'Чаплыгинский',
           'доброе': 'Добровский', 'тербуны': 'Тербунский', 'хлевное': 'Хлевенский', 'становое': 'Становлянский', 
           'долгоруково': 'Долгоруковский', 'лев-толстой': 'Лев-Толстовский', 'лев толстой': 'Лев-Толстовский',
           'измалково': 'Измалковский', 'волово': 'Воловский', 'красное': 'Краснинский', 
           }

Муниципальное образование	Все насел
Липецкая область	1 139 371
Городской округ г. Липецк	508 573
Городской округ г. Елец	102 313
Грязинский муниципальный район	79 548
Липецкий муниципальный район	52 690
Усманский муниципальный район	49 552
Лебедянский муниципальный район	38 096
Задонский муниципальный район	33 872
Добринский муниципальный район	32 906
Данковский муниципальный район	30 466
Чаплыгинский муниципальный район	29 769
Елецкий муниципальный район	28 415
Добровский муниципальный район	23 623
Тербунский муниципальный район	21 651
Хлевенский муниципальный район	18 697
Становлянский муниципальный район	16 995
Долгоруковский муниципальный район	16 498
Лев-Толстовский муниципальный район	16 197
Измалковский муниципальный район	15 250
Воловский муниципальный район	12 161
Краснинский муниципальный район	12 099

#### Распознавание адресов

In [5]:
print('Обработка mother_address...')
start_time = datetime.now()
for i in df.index[:]:
    for mo in mo_dict.keys():        
        if df.loc[i, 'mother_address'].lower().find(mo) != -1:
            df.loc[i, 'mo'] = mo_dict[mo]
            break
        else:
            df.loc[i, 'mo'] = 'Адрес матери не указан или за пределами Липецкой области'
print(f'Elapsed time {datetime.now() - start_time}')
print(f'Количество записей: {len(df)}')

Обработка mother_address...
Elapsed time 0:03:57.697987
Количество записей: 43367


#### Строки с адресами за пределами Липецкой области или без адреса

In [6]:
print(f"Количество нераспознанных адресов: {len(df[df.mo.isna()][['mother_address', 'mo']][:])}")
df[df.mo=='Адрес матери не указан или за пределами Липецкой области'].mother_address.unique()

Количество нераспознанных адресов: 0


array(['', 'Тамбовская обл Петровское с 1-я Восточная ул 35',
       'Курск г Энгельса ул 1-35',
       'Липецкая, Александровка, Широкая ул., д. 3/3, кв. 3', '29 7',
       'Калужская обл Обнинск г Аксенова ул 15-119',
       'Волгоградская обл Суровикино г Ленина ул 248',
       'Мордово рп Ворошилова ул 31', 'Воронеж г Победы б-р 22-159',
       'Тамбовская обл Мичуринск г Чайкиной ул 35',
       'Тамбовская обл Петровский р-н Крутое с 2',
       'Петровский р-н Знаменка с Набережная ул 7',
       'Тюменская, Заводоуковск, Новая ул., д. 41',
       'Воронежская обл Рамонский р-н Горожанка с Центральная ул 43',
       'Мордово рп Октябрьская ул 33 1',
       'Петровское с Революционная ул 37 1', 'Москва г Широкая ул 16 90',
       'Тамбовская обл Мичуринский р-н Коминтерн п Мичурина ул 23',
       '394062 Воронеж г Антокольского ул 10 40',
       '393070 Петровское с Луговая ул 34 1',
       '393070 Петровский р-н Добровольский п',
       'Тамбовская обл Уваровский р-н Нижний Шибряй 

#### Финальная обработка названий районов

In [7]:
print(f'Количество районов: {len(df.mo.unique())}')
print(sorted(df.mo.unique()))

Количество районов: 21
['Адрес матери не указан или за пределами Липецкой области', 'Воловский', 'Грязинский', 'Данковский', 'Добринский', 'Добровский', 'Долгоруковский', 'Елец', 'Елецкий', 'Задонский', 'Измалковский', 'Краснинский', 'Лебедянский', 'Лев-Толстовский', 'Липецк', 'Липецкий', 'Становлянский', 'Тербунский', 'Усманский', 'Хлевенский', 'Чаплыгинский']


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43367 entries, 0 to 43366
Data columns (total 14 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   organization                43367 non-null  object        
 1   mother_address              43367 non-null  object        
 2   mother_date_of_birth        43367 non-null  datetime64[ns]
 3   mother_year_of_birth        43367 non-null  int64         
 4   marital_status              43367 non-null  object        
 5   education                   43367 non-null  object        
 6   due_date                    43367 non-null  datetime64[ns]
 7   due_date_year               43367 non-null  int64         
 8   due_date_month              43367 non-null  int64         
 9   sex                         43367 non-null  object        
 10  which_born_multiple_births  930 non-null    float64       
 11  which_born                  43367 non-null  int64     

In [10]:
save_to_excel=True
if save_to_excel:
    print('Сохраняем данные в файл...')
    path = r'Рассчеты/'
    with pd.ExcelWriter(f'{path}fertility_preprocessed.xlsx', engine='openpyxl') as writer:
        df.to_excel(writer, sheet_name=f'fertility_preprocessed', header=True, index=False, encoding='1251')

Сохраняем данные в файл...


In [11]:
df

,organization,mother_address,mother_date_of_birth,mother_year_of_birth,marital_status,education,due_date,due_date_year,due_date_month,sex,which_born_multiple_births,which_born,mother_age,mo
0,"Государственное учреждение здравоохранения ""Те...",399540 Тербуны с Бунина ул 22,1995-03-21 03:00:00,1995,Состоит в зарегистрированном браке,Среднее (полное) общее,2017-01-01 00:00:00,2017,1,М,NaN,1,21,Тербунский
1,"Государственное учреждение здравоохранения ""Те...",398908 Липецк г Моршанская ул 6 а 49,1996-11-19 03:00:00,1996,Состоит в зарегистрированном браке,,2017-01-01 08:30:00,2017,1,М,NaN,1,20,Липецк
2,"Государственное учреждение здравоохранения ""Ли...",398901 Липецк г Шаумяна ул 15,1997-03-04 03:00:00,1997,Состоит в зарегистрированном браке,Среднее профессиональное,2017-01-01 20:05:00,2017,1,Ж,NaN,1,19,Липецк
3,"Государственное учреждение здравоохранения ""Ли...",Липецк г сдт Дачный-3 тер 113,1987-03-17 03:00:00,1987,Состоит в зарегистрированном браке,Высшее профессиональное,2017-01-02 06:15:00,2017,1,Ж,NaN,2,29,Липецк
4,"Государственное учреждение здравоохранения ""Ли...",Грязинский р-н Большой Самовец с Новая ул 35,1989-01-04 03:00:00,1989,Состоит в зарегистрированном браке,Высшее профессиональное,2017-01-01 04:15:00,2017,1,М,NaN,2,27,Грязинский
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43362,"Государственное учреждение здравоохранения ""Хл...",399255 Хлевенский р-н Муравьевка с Дорожная ул...,1996-03-14 03:00:00,1996,Состоит в зарегистрированном браке,Неполное высшее,2021-04-19 16:15:00,2021,4,Ж,NaN,1,25,Хлевенский
43363,"Государственное учреждение здравоохранения ""Те...",399580 Волово с Комсомольская ул 86,1983-09-13 03:00:00,1983,Состоит в незарегистрированном браке,Среднее общее,2021-04-19 17:05:00,2021,4,Ж,NaN,3,37,Воловский
43364,"Государственное учреждение здравоохранения ""Ли...",Тербуны с Липецкая ул 7,1985-08-07 03:00:00,1985,Состоит в зарегистрированном браке,Высшее профессиональное,2021-04-19 18:00:00,2021,4,М,NaN,3,35,Тербунский
43365,"Государственное учреждение здравоохранения ""Ли...",399061 Грязи г Заречная ул 33,1992-07-24 03:00:00,1992,Состоит в незарегистрированном браке,Среднее профессиональное,2021-04-19 20:30:00,2021,4,М,NaN,2,28,Грязинский


In [12]:
table=pd.pivot_table(df, 
                     values=['organization'], 
                     index=['mo'], columns=['due_date_year'], 
                     aggfunc='count', 
                     fill_value=0.0, margins=True, dropna=True, 
                     margins_name='Итого', observed=False)
table.sort_values(by=[('organization', 'Итого')], ascending=False, inplace=True)
table

organization               \
due_date_year                                              2017   2018  2019   
mo                                                                             
Итого                                                     11809  11004  9799   
Липецк                                                     5344   5023  4554   
Елец                                                        938    802   748   
Грязинский                                                  742    805   670   
Адрес матери не указан или за пределами Липецко...          837    609   457   
Липецкий                                                    517    527   425   
Усманский                                                   384    396   324   
Лебедянский                                                 389    371   336   
Данковский                                                  299    267   252   
Добринский                                                  241    249   256   
Чаплыгинский                                                254    270   239   
Задонский                                                   279    243   211   
Елецкий                                                     241    235   204   
Добровский                                                  225    217   186   
Тербунский                                                  199    172   166   
Хлевенский                                                  175    143   111   
Лев-Толстовский                                             161    157   122   
Становлянский                                               135    129   126   
Долгоруковский                                              130    111   121   
Измалковский                                                118    108   123   
Краснинский                                                 116     95    91   
Воловский                                                    85     75    77   

                                                                       
due_date_year                                       2020  2021  Итого  
mo                                                                     
Итого                                               8837  1918  43367  
Липецк                                              3982   862  19765  
Елец                                                 655   137   3280  
Грязинский                                           655   165   3037  
Адрес матери не указан или за пределами Липецко...   365    72   2340  
Липецкий                                             399    94   1962  
Усманский                                            335    65   1504  
Лебедянский                                          276    65   1437  
Данковский                                           238    45   1101  
Добринский                                           219    56   1021  
Чаплыгинский                                         201    36   1000  
Задонский                                            195    44    972  
Елецкий                                              199    40    919  
Добровский                                           200    38    866  
Тербунский                                           155    25    717  
Хлевенский                                           144    26    599  
Лев-Толстовский                                      119    38    597  
Становлянский                                        102    23    515  
Долгоруковский                                       120    25    507  
Измалковский                                         117    24    490  
Краснинский                                           87    24    413  
Воловский                                             74    14    325